In [ ]:
import polars as pl, matplotlib.pyplot as plt, datetime, matplotlib.dates as mdates, matplotlib.ticker as mticker, geopandas as gpd, matplotlib.colors as colors, contextily as ctx, pandas as pd, numpy as np, statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from matplotlib.cm import ScalarMappable
from matplotlib.colors import Normalize
from matplotlib.ticker import AutoMinorLocator
from statsmodels.nonparametric.smoothers_lowess import lowess
from matplotlib.patches import Patch
from xgboost import XGBRegressor
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
data = '../data/london_burglaries.parquet'
burglaries = pd.read_parquet(data)
lsoa_gdf = gpd.read_file('../data/geo/london_lsoa.gpkg')
resi = pl.read_csv("../data/geo/lsoa_residential_classification_2018.csv")
imd = pl.read_parquet("../data/imd_2019.parquet")
imd.head()

In [ ]:
# Count burglaries per LSOA
burglary_counts = burglaries.groupby('LSOA code').size().reset_index(name='burglary_count')

# Merge the count data with the geographic data
lsoa_with_counts = lsoa_gdf.merge(burglary_counts, left_on='LSOA21CD', right_on='LSOA code', how='left')

# Fill NaN values with 0 (LSOAs with no recorded burglaries)
lsoa_with_counts['burglary_count'] = lsoa_with_counts['burglary_count'].fillna(0)
lsoa_with_counts['burglary_count_log'] = np.log2(burglary_counts['burglary_count'])
lsoa_with_counts

In [ ]:
# Set figure background to dark
fig, ax = plt.subplots(1, 1, figsize=(15, 10), facecolor='white', dpi=300)
ax.set_facecolor('#121212')  # Dark background for the plotting area

# Create a custom colormap for better visualization
vmin = lsoa_with_counts['burglary_count'].min()
vmax = lsoa_with_counts['burglary_count'].max()

# Plot with normal scale
lsoa_map = lsoa_with_counts.plot(
    column='burglary_count',
    ax=ax,
    cmap='inferno',
    norm=colors.LogNorm(vmin=max(1, vmin), vmax=vmax),
    legend=False,
    legend_kwds={'label': 'Number of Burglaries (log scale)'}
)

# Add dark basemap
ctx.add_basemap(
    ax,
    source=ctx.providers.CartoDB.Positron,
    crs=lsoa_with_counts.crs.to_string()
)

# Create a custom colorbar/legend
sm = ScalarMappable(norm=colors.LogNorm(vmin=max(1, vmin), vmax=vmax), cmap='inferno')
sm.set_array([])
cbar = fig.colorbar(sm, ax=ax, orientation='vertical', pad=0.01)
cbar.set_label('Number of Burglaries (log scale)', color='black', size=12)
cbar.ax.yaxis.set_tick_params(color='white')
plt.setp(plt.getp(cbar.ax.axes, 'yticklabels'), color='black')

# Add title and labels
# plt.title('Burglaries per LSOA in London (2011-12 - 2025-02) log scaled', fontsize=16,color='black')
plt.axis('off')

# Save the plot
plt.savefig('../figs/london_burglaries_per_lsoa_log_scaled.png', dpi=300, bbox_inches='tight', facecolor='white')

# Show the plot
plt.show()

# Optional: Create a table of top 10 LSOAs with highest burglary counts
top_lsoas = lsoa_with_counts.sort_values('burglary_count', ascending=False).head(10)
top_lsoas_table = top_lsoas[['LSOA21CD', 'LSOA21NM', 'burglary_count']]
print("\nTop 10 LSOAs with highest burglary counts:")
print(top_lsoas_table)

In [ ]:
# ---------- FIGURE & AXES (defaults give a white background) ---------------
fig, ax = plt.subplots(1, 1, figsize=(15, 10), dpi=300)

# ---------- COLOUR MAP (linear scale) --------------------------------------
vmin = lsoa_with_counts['burglary_count'].min()
vmax = lsoa_with_counts['burglary_count'].max()
norm = Normalize(vmin=vmin, vmax=vmax)

lsoa_with_counts.plot(column='burglary_count', ax=ax, cmap='inferno', norm=norm, legend=False)

# ---------- LIGHT BASEMAP ---------------------------------------------------
ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron, crs=lsoa_with_counts.crs.to_string())

# ---------- COLOUR BAR / LEGEND --------------------------------------------
sm = ScalarMappable(norm=norm, cmap='inferno')
sm.set_array([])
cbar = fig.colorbar(sm, ax=ax, orientation='vertical', pad=0.01)
cbar.set_label('Number of Burglaries (linear scale)', size=12)
cbar.ax.yaxis.set_tick_params(color='black')
plt.setp(cbar.ax.get_yticklabels(), color='black')

# ---------- TITLE & CLEAN-UP ------------------------------------------------
# plt.title(
#     'Burglaries per LSOA in London (2011-12 – 2025-02), linear scale',
#     fontsize=16,
#     color='black'
# )
plt.axis('off')

# ---------- SAVE & SHOW -----------------------------------------------------
plt.savefig('../figs/london_burglaries_per_lsoa_linear_scaled.png', dpi=300, bbox_inches='tight')
plt.show()

# ---------- OPTIONAL: Top-10 LSOAs -----------------------------------------
top_lsoas = (
    lsoa_with_counts.sort_values('burglary_count', ascending=False)
                    .head(10)[['LSOA21CD', 'LSOA21NM', 'burglary_count']]
)
print("\nTop 10 LSOAs with highest burglary counts:")
print(top_lsoas)


In [ ]:
# Load and sort
crimes = pl.read_parquet(data).sort("Month")

# --- Full counts by month ---------------------------------------------------
crimes_count = crimes.group_by("Month").agg(pl.len()).sort("Month")

# --- Residential-only counts by month ---------------------------------------
res_crimes = crimes.filter(pl.col("LSOA code").is_in(residential_codes))
res_crimes_count = res_crimes.group_by("Month").agg(pl.len()).sort("Month")

# --- Convert month strings to datetime --------------------------------------
date_list = [datetime.datetime.strptime(x, "%Y-%m") for x in crimes_count["Month"].to_list()]
res_date_list = [datetime.datetime.strptime(x, "%Y-%m") for x in res_crimes_count["Month"].to_list()]

# --- Convert for plotting
x = mdates.date2num(date_list)
x_res = mdates.date2num(res_date_list)
y = crimes_count['len'].to_numpy()
y_res = res_crimes_count["len"].to_numpy()

# --- Normalize both series by their own max
y_norm = y / y.max()
y_res_norm = y_res / y_res.max()

# --- Smooth normalized values
smooth_y = lowess(y_norm, x, frac=0.10, return_sorted=False)
smooth_y_res = lowess(y_res_norm, x_res, frac=0.10, return_sorted=False)

# --- COVID lockdown periods
highlight_periods = [
    (datetime.datetime(2020, 3, 23), datetime.datetime(2020, 7, 4)),
    (datetime.datetime(2020, 11, 5), datetime.datetime(2020, 12, 2)),
    (datetime.datetime(2021, 1, 4), datetime.datetime(2021, 3, 8)),
]

# --- Plotting ----------------------------------------------------------------
fig, ax = plt.subplots(figsize=(16, 8), facecolor='white', dpi=300)
ax.set_facecolor("white")

# Full series
ax.plot(x, y_norm, marker='o', markersize=4, color='#DC267F', alpha=0.3, label='All LSOAs')
ax.plot(x, smooth_y, linewidth=2, color='#61BBF7', alpha=0.8, label='All LSOAs (smoothed)')

# Residential series
ax.plot(x_res, y_res_norm, marker='o', markersize=4, linestyle='-', color='#FE6100', alpha=0.6, label='Residential LSOAs')
ax.plot(x_res, smooth_y_res, linewidth=2, linestyle='-', color='#993CEC', alpha=0.9, label='Residential (smoothed)')

# Highlight lockdowns
for i, (start, end) in enumerate(highlight_periods):
    ax.axvspan(mdates.date2num(start), mdates.date2num(end), color='#FFB000', alpha=0.25,
               label='COVID lockdowns' if i == 0 else None)

# Axes formatting
ax.xaxis.set_major_locator(mdates.YearLocator())
ax.xaxis.set_major_formatter(mdates.DateFormatter("%Y"))
ax.xaxis.set_minor_locator(mdates.MonthLocator(interval=1))
ax.yaxis.set_minor_locator(AutoMinorLocator())

# Grids and spines
ax.grid(True, which='major', linestyle='--', linewidth=0.5, color='gray')
ax.grid(True, which='minor', linestyle='--', linewidth=0.5, color='#e3e3e3')
for spine in ax.spines.values():
    spine.set_edgecolor('black')
    spine.set_linewidth(1)

# Labels and legend
plt.xlabel("Month", fontweight='bold', fontsize=14)
plt.ylabel("Normalized burglary count", fontweight='bold', fontsize=14)
plt.title("Normalized burglary trends per month (2011/12 - 2025/02)", fontweight='bold', fontsize=16)
plt.xticks(rotation=0)
ax.legend(loc='upper right')

# Save and show
plt.savefig('../figs/monthly_burglary_counts_normalized.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Filter for residential-dominant LSOAs
residential_lsoas = resi.filter(pl.col("is_residential_dominant") == True)
residential_codes = residential_lsoas.select("LSOA21CD").to_series().to_list()

print(f"Total LSOAs in London: {len(lsoa_gdf)}")
print(f"Residential-dominant LSOAs: {len(residential_codes)}")
print(f"Percentage residential: {len(residential_codes)/len(lsoa_gdf)*100:.1f}%")

# Merge residential data with geographic data
resi_pandas = resi.to_pandas()
london_gdf_merged = lsoa_gdf.merge(resi_pandas[['LSOA21CD', 'residential_pct_2018', 'is_residential_dominant']],
                                     on='LSOA21CD', how='left')

# Create the map
fig, ax = plt.subplots(1, 1, figsize=(12, 10), dpi=300)

# Plot all LSOAs in light gray (non-residential or missing data)
london_gdf_merged[london_gdf_merged['is_residential_dominant'] != True].plot(
    ax=ax, color='lightgray', edgecolor='white', linewidth=0.1, alpha=0.7)

# Plot residential-dominant LSOAs with intensity based on residential_pct_2018
residential_gdf = london_gdf_merged[london_gdf_merged['is_residential_dominant'] == True]
residential_gdf.plot(ax=ax, column='residential_pct_2018', cmap='Blues',
                    edgecolor='white', linewidth=0.1, legend=True,
                    legend_kwds={'label': 'Residential %', 'shrink': 0.8})

# Styling
ax.set_title('London LSOAs: Residential-Dominant Areas', fontsize=16, fontweight='bold', pad=20)
ax.axis('off')

# Add legend
legend_elements = [
    Patch(facecolor='lightgray', alpha=0.7, label=f'Non-Residential ({len(lsoa_gdf) - len(residential_codes):,})'),
]
ax.legend(handles=legend_elements, loc='lower left', fontsize=12)

# Add statistics text box
stats_text = f"""Dataset Coverage:
• Total LSOAs: {len(lsoa_gdf):,}
• Residential-Dominant: {len(residential_codes):,}
• Percentage: {len(residential_codes)/len(lsoa_gdf)*100:.1f}%"""

ax.text(0.02, 0.98, stats_text, transform=ax.transAxes, fontsize=10,
        verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))

plt.tight_layout()
plt.show()

# Additional analysis
print("\nResidential Classification Summary:")
residential_stats = resi.select([
    pl.col("residential_pct_2018").mean().alias("avg_residential_pct"),
    pl.col("residential_pct_2018").median().alias("median_residential_pct"),
    pl.col("is_residential_dominant").sum().alias("count_residential"),
    pl.len().alias("total_count")
])

print(f"Average residential percentage: {residential_stats['avg_residential_pct'][0]:.1f}%")
print(f"Median residential percentage: {residential_stats['median_residential_pct'][0]:.1f}%")

# Show distribution of residential advantage
residential_advantage_stats = residential_lsoas.select([
    pl.col("residential_advantage").mean().alias("mean_advantage"),
    pl.col("residential_advantage").min().alias("min_advantage"),
    pl.col("residential_advantage").max().alias("max_advantage")
])

print(f"\nResidential Advantage Statistics:")
print(f"Mean advantage: {residential_advantage_stats['mean_advantage'][0]:.1f}%")
print(f"Range: {residential_advantage_stats['min_advantage'][0]:.1f}% to {residential_advantage_stats['max_advantage'][0]:.1f}%")

In [ ]:
temporal_features = pl.read_parquet("../data/features.parquet")
random_lsoa = np.random.choice(temporal_features["LSOA code"].unique())
print("Random LSOA chosen:", random_lsoa)

sel = (
    temporal_features
        .filter(pl.col("LSOA code") == random_lsoa)
        .sort(["year", "month"])
)
train_dates     = pd.to_datetime({"year": sel["year"], "month": sel["month"], "day": 1})
burglary_counts = sel["burglary_count"].to_numpy()
hma_4           = sel["burglary_count_hma_4"].to_numpy().ravel()
hma_5           = sel["burglary_count_hma_5"].to_numpy().ravel()
tema_6          = sel["burglary_count_tema_6"].to_numpy().ravel()
ewm_6           = sel["burglary_count_ewm_6"].to_numpy().ravel()

In [ ]:
fig, ax1 = plt.subplots(1, 1, figsize=(16, 8), dpi=300)

# Plot burglary counts on left y-axis
color = 'tab:red'
ax1.set_xlabel('Year')
ax1.set_ylabel('Burglary counts', color=color)
ax1.plot(train_dates, burglary_counts, 'r-', linewidth=1.5, label='Burglary counts', marker='o', markersize=4)
ax1.plot(train_dates, hma_5, 'b--', linewidth=1, label='HMA 5', marker='o', markersize=4, alpha=0.2)
ax1.plot(train_dates, tema_6, 'g-', linewidth=1, label='TEMA 6', marker='o', markersize=4, alpha=0.2)
ax1.plot(train_dates, ewm_6, 'c-', linewidth=1, label='EWM 6', marker='o', markersize=4, alpha=0.2)
ax1.tick_params(axis='y', labelcolor=color)

# Format x-axis
ax1.xaxis.set_major_locator(mdates.MonthLocator(bymonth=1, interval=1))
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
ax1.xaxis.set_minor_locator(mdates.MonthLocator())
fig.autofmt_xdate(ha='center', rotation=0)

ax1.set_title(f'LSOA {random_lsoa} – Burglary vs HMA-3')
# --- x-axis locators -------------------------------------------------------
ax1.xaxis.set_major_locator(mdates.YearLocator())           #  1-Jan every year
ax1.xaxis.set_major_formatter(mdates.DateFormatter("%Y"))   # optional: show just the year
ax1.xaxis.set_minor_locator(mdates.MonthLocator(interval=1))  # every month

ax1.yaxis.set_minor_locator(AutoMinorLocator())

# --- gridlines -------------------------------------------------------------
ax1.grid(True, which='major', linestyle='--', linewidth=0.5, color='gray')
ax1.grid(True, which='minor', linestyle='--', linewidth=0.5, color='#e3e3e3')
ax1.grid(True, which='minor', axis='y', linestyle='--', linewidth=0.5, color='#e3e3e3')

# **Add a black outline (border) to the plot by customizing the spines**
for spine in ax1.spines.values():
    spine.set_edgecolor('black')
    spine.set_linewidth(1)

ax1.legend(loc='upper right')

# Rotate x-axis tick labels
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

In [ ]:
X_train = pl.read_parquet("../data/X_train.parquet").drop("LSOA code").to_numpy()
y_train = pl.read_parquet("../data/y_train.parquet").to_numpy().ravel()
X_test  = pl.read_parquet("../data/X_test.parquet").drop("LSOA code").to_numpy()
y_test  = pl.read_parquet("../data/y_test.parquet").to_numpy().ravel()

optimized_model = XGBRegressor(
    colsample_bytree=0.679486272613669,
    learning_rate=0.16678305712187014,
    max_depth=4,
    min_child_weight=8,
    n_estimators=468,
    subsample=0.9085081386743783,
    objective="count:poisson",
    eval_metric="rmse",
    tree_method="hist",
    random_state=42,
    early_stopping_rounds=72,
    n_jobs=-1
)

adjusted_model = optimized_model.fit(
    X_train, y_train,
    eval_set=[(X_test, y_test)],
    verbose=False
)

train_df = pl.read_parquet("../data/X_train.parquet").with_columns(pl.read_parquet("../data/y_train.parquet"))
test_df = pl.read_parquet("../data/X_test.parquet").with_columns(pl.read_parquet("../data/y_test.parquet"))

# Get predictions
train_predictions = adjusted_model.predict(X_train)
test_predictions = adjusted_model.predict(X_test)

# Add predictions to dataframes
train_df = train_df.with_columns(pl.Series("predictions", train_predictions))
test_df = test_df.with_columns(pl.Series("predictions", test_predictions))

train_avg = (
    train_df
    .group_by(["time_index_norm", "year", "month"])
    .agg([
        pl.col("burglary_count").mean().alias("avg_actual"),
        pl.col("predictions").mean().alias("avg_predicted")
    ])
    .sort("time_index_norm")
)

test_avg = (
    test_df
    .group_by(["time_index_norm", "year", "month"])
    .agg([
        pl.col("burglary_count").mean().alias("avg_actual"),
        pl.col("predictions").mean().alias("avg_predicted")
    ])
    .sort("time_index_norm")
)

train_dates = pd.to_datetime({"year": train_avg["year"], "month": train_avg["month"], "day": 1})
test_dates  = pd.to_datetime({"year": test_avg["year"],  "month": test_avg["month"],  "day": 1})

train_actual_avg = train_avg["avg_actual"].to_list()
train_pred_avg   = train_avg["avg_predicted"].to_list()
test_actual_avg  = test_avg["avg_actual"].to_list()
test_pred_avg    = test_avg["avg_predicted"].to_list()

fig, ax = plt.subplots(figsize=(16, 8), dpi=300)

ax.plot(train_dates, train_actual_avg,  'b-',   linewidth=2, label='Training Actual (Avg)', marker='o', markersize=4)
ax.plot(train_dates, train_pred_avg,    'r--',  linewidth=1, label='Training Predicted (Avg)')
ax.plot(test_dates,  test_actual_avg,   'g-',   linewidth=2, label='Test Actual (Avg)',     marker='o', markersize=4)
ax.plot(test_dates,  test_pred_avg,     'r--',  linewidth=1, label='Test Predicted (Avg)')

# vertical split line at the boundary between the two date series
ax.axvline(train_dates.iloc[-1] + pd.Timedelta(days=15), color='black',
           linestyle=':', alpha=0.7, linewidth=2, label='Train/Test Split')

# date-axis formatting
ax.xaxis.set_major_locator(mdates.MonthLocator(bymonth=1, interval=1))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
ax.xaxis.set_minor_locator(mdates.MonthLocator())
fig.autofmt_xdate(ha='center', rotation=0)

ax.set_title('Average Burglary Count Across All LSOAs\n(Training: 111 months, Testing: 48 months)',
             fontsize=16, fontweight='bold')
ax.set_xlabel('Year', fontsize=12)
ax.set_ylabel('Average Burglary Count per LSOA', fontsize=12)
ax.legend(fontsize=11)
ax.grid(True, alpha=0.4)
ax.grid(True, which='major', linestyle='-',  linewidth=0.8, alpha=0.4)
ax.grid(True, which='minor', linestyle='--', linewidth=0.6, alpha=0.3)
ax.tick_params(axis='both', which='minor', length=0)
ax.yaxis.set_minor_locator(mticker.AutoMinorLocator())

# Add some statistics as text
train_rmse = np.sqrt(np.mean((np.array(train_actual_avg) - np.array(train_pred_avg))**2))
test_rmse = np.sqrt(np.mean((np.array(test_actual_avg) - np.array(test_pred_avg))**2))

plt.text(0.02, 0.98, f'Training Avg RMSE: {train_rmse:.3f}\nTesting Avg RMSE: {test_rmse:.3f}',
         transform=plt.gca().transAxes, fontsize=10,
         bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8),
         verticalalignment='top')

plt.tight_layout()
plt.show()

In [ ]:
# 1. LOAD & AGGREGATE BURGLARIES
burg_path = "../data/london_burglaries.parquet"
lsoa_col  = "LSOA code"

burg_df = (
    pl.read_parquet(burg_path)
      # keep only residential LSOAs
      .filter(pl.col(lsoa_col).is_in(residential_codes))
      .group_by(lsoa_col)
      .agg(pl.len().alias("burglary_count"))
      .with_columns(
          # ln(1 + count)  → keeps zeros, avoids –inf
          (pl.col("burglary_count").cast(pl.Float64) + 1)
              .log()
              .alias("log_burglary_count")
      )
)

# 2. LOAD IMD 2019 AND JOIN
imd_path = "../data/imd_2019.parquet"
imd_df   = pl.read_parquet(imd_path).rename({lsoa_col: lsoa_col})

combined = burg_df.join(imd_df, on=lsoa_col, how="left")

# 3.  BUILD MODELLING FRAME
base_vars = [
    "Living Environment Score",
    "Crime Score",
    "Education, Skills and Training Score",
    "Income Score (rate)",
]

interaction_vars = ["Crime_Education"]
numeric_vars     = base_vars + ["log_burglary_count"]

model_df = (
    combined
      .with_columns([
          (pl.col("Crime Score") * pl.col("Education, Skills and Training Score"))
              .alias("Crime_Education"),
      ])
      .select([lsoa_col] + numeric_vars)
      .with_columns([pl.col(col).cast(pl.Float64) for col in numeric_vars])
      .drop_nulls(subset=numeric_vars)
      .filter(~pl.any_horizontal(pl.col(numeric_vars).is_infinite()))
)

print(f"Rows after cleaning (residential only): {model_df.height:,}")

# 4.  OLS WITH HC3 ROBUST SEs
df      = model_df.to_pandas()
X_vars  = base_vars
X       = sm.add_constant(df[X_vars])
y       = df["log_burglary_count"]

ols = sm.OLS(y, X).fit(cov_type="HC3")
print(ols.summary())

# 5. VIFs
vif = pd.Series(
    [variance_inflation_factor(X.values, i) for i in range(X.shape[1])],
    index=["const"] + X_vars, name="VIF"
)
print("\nVariance-Inflation Factors:")
print(vif.round(3))


In [ ]:
# pull objects from previous cell
y_actual = y                     # Series of actual log-burglary counts
y_pred   = ols.fittedvalues      # fitted values from statsmodels

# calibration line: ŷ = a + b·y
cal_model = sm.OLS(y_pred, sm.add_constant(y_actual)).fit()

x_line   = np.linspace(y_actual.min(), y_actual.max(), 400)
pred     = cal_model.get_prediction(sm.add_constant(x_line))
ci_lower, ci_upper = pred.conf_int().T
y_line             = pred.predicted_mean

plt.figure(figsize=(7, 7), dpi=300)

# scatter
plt.scatter(y_actual, y_pred, alpha=0.7, s=22, label="LSOAs", color='#1E88E5')

# 45-degree perfect-fit line
lims = [min(y_actual.min(), y_pred.min()), max(y_actual.max(), y_pred.max())]
plt.plot(lims, lims, linestyle="--", linewidth=1.2, color='#D81B60', label="Perfect fit (45°)")

# calibration line + 95 % CI
r2 = ols.rsquared
plt.plot(x_line, y_line, linewidth=2.5, label=f"OLS line (R² = {r2:.3f})", color='#FFC107')
plt.fill_between(x_line, ci_lower, ci_upper, alpha=0.25, label="95 % CI", color='#FFC107')

# cosmetics
plt.title("Actual vs Fitted – Residential LSOAs", fontsize=14, fontweight="bold", pad=15)
plt.xlabel("Actual burglary count (log)", fontweight="bold")
plt.ylabel("Fitted value (OLS)",        fontweight="bold")

# thicker axes
for spine in plt.gca().spines.values():
    spine.set_linewidth(1.2)

plt.grid(alpha=0.5, linestyle="--", linewidth=0.5)
plt.legend(frameon=False)
plt.tight_layout()
plt.savefig("../figs/actual_vs_fitted_OLS.png", dpi=300)
plt.show()